In [317]:
# imports
import tkinter as tk
from PIL import ImageGrab
from random import randint, random
from time import sleep
from numpy.random import choice
import numpy as np

In [318]:
# ----------------------------------------
# constants
MIN_WIDTH = 50
MAX_WIDTH = 700

MIN_HEIGHT = 50
MAX_HEIGHT = 300

MERGE_RATE_EDGE = 0.35
MERGE_RATE = 0.3

# WIDGET_LIST = ["empty", "button", "label", "entry"]
# WIDGET_PROB = [0.35,0.15,0.35,0.15]

WIDGET_LIST = ["empty", "label", "button"]
WIDGET_PROB = [0.0, 0.5, 0.5]

In [319]:
# ----------------------------------------
# the widget grid
# ----------------------------------------

import tkinter as tk

root = tk.Tk()
screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()
root.destroy()
root.mainloop()

# generating rows and columns

# columns
row_coords = [0]
column_coords = [0]

while 1:
    new_x = randint(MIN_WIDTH, MAX_WIDTH)

    if (row_coords[-1] + new_x) > screen_width:
        row_coords.append( screen_width)
        break
    else:
        row_coords.append( row_coords[-1] + new_x)

while 1:
    new_y = randint(MIN_HEIGHT, MAX_HEIGHT)

    if (column_coords[-1] + new_y) > screen_height:
        column_coords.append( screen_height)
        break
    else:
        column_coords.append( column_coords[-1] + new_y)

print(row_coords)
print(column_coords)

[0, 364, 732, 1253, 1408, 1536]
[0, 182, 330, 484, 688, 864]


In [320]:
class cell:

    def __init__(self, lx, rx, uy, dy, ):

        self.lx = lx
        self.rx = rx
        self.uy = uy
        self.dy = dy

        self.edge = (lx == 0) or (uy == 0) or\
                    (rx == screen_width) or (dy == screen_height)
        
        self.active = True
        self.merged = False
        self.widget = ""
        self.neighbours = []

In [321]:
# creating cells within a grid

grid = []
cell_list = []

for i,r in enumerate(row_coords[:-1]):
    grid.append([])
    for j,c in enumerate(column_coords[:-1]):

        new_cell = cell(r, row_coords[i+1],
                        c,column_coords[j+1])
        
        grid[-1].append(new_cell)
        cell_list.append(new_cell)

In [322]:
# finding neighbours and merging

max_row = len(grid) - 1
max_col = len(grid[0]) - 1


def add_neighbours(c):
    if row > 0:
        c.neighbours.append( grid[row-1][col])
    if row < max_row:
        c.neighbours.append( grid[row+1][col])
    if col > 0:
        c.neighbours.append( grid[row][col-1])
    if col < max_col:
        c.neighbours.append( grid[row][col+1])
    return

def merge_cells(c):
    for n in c.neighbours:
        if (not c.merged) and (not n.merged):
            merge_chance = random()

            if merge_chance < MERGE_RATE:
                n.active = False
                n.merged = True
                c.merged = True
                c.lx = min( c.lx, n.lx)
                c.lx = min( c.uy, n.uy)
                c.lx = max( c.rx, n.rx)
                c.lx = max( c.dy, n.dy)
    return

def assign_widget(c):
    widget = choice( WIDGET_LIST, 1, p=WIDGET_PROB)
    c.widget = widget[0]
    return

for row,_ in enumerate(grid):
    for col,_ in enumerate(grid[row]):
        c = grid[row][col]

        if c.active:
            add_neighbours(c)
            merge_cells(c)
            assign_widget(c)

In [323]:
cell_list = []

for row,_ in enumerate(grid):
    for col,_ in enumerate(grid[row]):
        c = grid[row][col]

        if c.active:
            cell_list.append(c)

In [324]:
[c.widget for c in cell_list]

['button',
 'label',
 'button',
 'button',
 'label',
 'label',
 'label',
 'button',
 'button',
 'button',
 'button',
 'label',
 'button',
 'label',
 'button']

In [325]:
sleep(5)

In [326]:
class MainApplication(tk.Frame):
    def __init__(self, parent, *args, **kwargs):
        tk.Frame.__init__(self, parent, *args, **kwargs)
        self.parent = parent

        for c in cell_list:
            if c.widget == "button":
                tk.Button( self, text="button", width=c.rx-c.lx).place(x=c.lx,y=c.uy, height=c.dy-c.uy)
            if c.widget == "label":
                tk.Label( self, text="label", background="#666666").place(x=c.lx, y=c.uy, width=c.rx-c.lx, height=c.dy-c.uy)

def take_screenshot_and_quit(root):
    root.update()
    screenshot = ImageGrab.grab()
    screenshot.save("screenshots/screenshot.png")
    screenshot.close()
    root.destroy()

if __name__ == "__main__":
    root = tk.Tk()
    root.geometry(f"{screen_width}x{screen_height}")
    root.attributes("-fullscreen", True)
    root.attributes("-topmost", True)
    MainApplication(root).pack(side="top", fill="both", expand=True)

    root.after(500, lambda: take_screenshot_and_quit(root))

    root.mainloop()